<h1 id="tocheading">SVM (Support Vector Machine)</h1>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Входные данные

$2$-классовая $n$-мерная выборка размера $K$ с числовыми признаками. То есть полагаем, что все объекты выборки имеют вид:
$$(x_1, x_2, \dots, x_n, c)\text{, где }c\in\{-1,1\}$$

## Постановка задачи

Хотим разделить выборку какой-то $(n-1)$-мерной гиперплоскостью, чтобы
  1) по одну сторону от гиперплоскости лежали экземпляры первого класса, а по другую стороны экземпляры второго класса (в идеальном случае);

  2) гиперплоскость была "оптимальной".

Какая прямая среди $L_1$, $L_2$ и $L_3$ лучше разделяет выборку?

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Classifier.svg/1920px-Classifier.svg.png" alt="drawing" width="500"/>

_**Оптимальная разделяющая гиперплоскость**_ &ndash; такая гиперплоскость, у которой сумма расстояний до двух ближайших к ней точек, лежащих по разные стороны от неё, максимальна. Это определение следует из того, что мы полагаем, что максимизация зазора между классами способствует более уверенной классификации

Математически искомую гиперплоскость можно записать в следующем виде:
$$l(\mathbf{x})=\mathbf{w}\cdot \mathbf{x} +b$$
Тогда определить класс нового объекта становится просто:
$$c(\mathbf{x})=\mathrm{sgn}(\mathbf{w}\cdot \mathbf{x} + b)$$

Таким образом, задача формулируется так: "найти $\mathbf{w}$ и $b$ такие, чтобы гиперплоскость $l(\mathbf{x})=\mathbf{w}\cdot \mathbf{x} +b$ была _**оптимальной разделяющей гиперплоскостью**_"

## Математика

### Случай линейно-разделимой выборки

Назовём _**отступом**_ такое выражение (по сути &ndash; расстояние от точки до гиперплоскости по модулю):
$$M_i=с_i\cdot(\mathbf{w}\cdot\mathbf{x}_i + b)$$

Можно произвести нормировку отступов коэффициентом $\alpha$:
$$M_i=\alpha \cdot с_i\cdot(\mathbf{w}\cdot\mathbf{x}_i + b) = с_i\cdot((\alpha \cdot\mathbf{w})\cdot\mathbf{x}_i + (\alpha \cdot b))$$
так, чтобы отступы ближайших к гиперплоскости точек были равны $1$

Таким образом, суть задачи не меняется &ndash; мы всё так же ищем вектор $(\alpha \cdot\mathbf{w})$ и число $(\alpha \cdot b)$ (далее будем обозначать их просто как $\mathbf{w}$ и $b$) так, чтобы чтобы гиперплоскость $l(\mathbf{x})=\mathbf{w}\cdot \mathbf{x} +b$ была _**оптимальной разделяющей гиперплоскостью**_

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/SVM_margins.svg/1920px-SVM_margins.svg.png" alt="drawing" width="500"/>

Через ближайшие к гиперплоскости точки можно провести полосу, ширина которой будет равна:
$$\mathrm{pr}_{\mathbf{w}}\left(\mathbf{x_+}-\mathbf{x_-}\right)=\left(\mathbf{x_+}-\mathbf{x_-}\right)\cdot \frac{\mathbf{w}}{\lVert\mathbf{w}\rVert}=\frac{\mathbf{w}\cdot\mathbf{x_+} - \mathbf{w}\cdot\mathbf{x_-}}{\lVert\mathbf{w}\rVert}=\frac{\frac{M_+ - b}{c_+}-\frac{M_- - b}{c_-}}{\lVert\mathbf{w}\rVert}=\frac{\frac{1 - b}{1}-\frac{1 - b}{-1}}{\lVert\mathbf{w}\rVert}=\frac{1 - b+b+1}{\lVert\mathbf{w}\rVert}=\frac{2}{\lVert\mathbf{w}\rVert}$$
где $\mathbf{x_+}$ &ndash; ближайшая точка с "положительной" стороны гиперплоскости, т.е. представитель положительного класса ($c_+ = 1$), а $\mathbf{x_-}$ &ndash; ближайшая точка с "отрицательной" стороны гиперплоскости, т.е. отрицательный класс ($c_- = 1$)

Получается, чем меньше $\lVert\mathbf{w}\rVert$, тем больше ширина полосы

Таким образом, можно сформулировать задачу математически:
$$
\left\{\begin{aligned} 
    & \lVert\mathbf{w}\rVert^2\rightarrow\min &\\ 
    & M_i\geqslant 1,&\text{ где }i = 1,\dots,K
\end{aligned}\right. 
$$

### Случай линейно-неразделимой выборки

Если выборка линейно-неразделима, то, как ни крути, будут экземпляры какого-либо из классов, которые будут лежать не на "своей" стороне от гиперплоскости. Такое поведение будет означать, что не для всех экземпляров класса будет выполняться условие
$$M_i \geqslant 1, \text{ где } i = 1, \dots, K$$

Разрешим эту проблему, введя ошибку $\xi_i$. Тогда условие на отступы может быть записано как:
$$M_i \geqslant 1 - \xi_i, \text{ где } i = 1, \dots, K$$
Очевидно, что $\forall i\in\{1,\dots,K\} \: \xi_i \rightarrow 0$, т.к. мы хотим минимизировать ошибку

Тогда задача будет записана как ($C$ &ndash; выбранный нами гиперпараметр):
$$
\begin{array}{cccc}
    \left\{\begin{aligned} 
        & \lVert\mathbf{w}\rVert^2+C\cdot\sum_{j=1}^{K}\xi_j\rightarrow\min &\\ 
        & M_i\geqslant 1-\xi_i,&\text{ где }i = 1,\dots,K\\
        & \xi_i\geqslant 0,&\text{ где }i = 1,\dots,K
    \end{aligned}\right.
    & \Rightarrow &
    \left\{\begin{aligned} 
        & \lVert\mathbf{w}\rVert^2+C\cdot\sum_{j=1}^{K}\xi_j\rightarrow\min &\\ 
        & \xi_i\geqslant1-M_i,&\text{ где }i = 1,\dots,K\\
        & \xi_i\geqslant 0,&\text{ где }i = 1,\dots,K
    \end{aligned}\right.
    & \Rightarrow \\
    & \Rightarrow &
    \left\{\begin{aligned} 
        & \lVert\mathbf{w}\rVert^2+C\cdot\sum_{j=1}^{K}\xi_j\rightarrow\min &\\ 
        & \xi_i=\max(0,1-M_i),&\text{ где }i = 1,\dots,K
    \end{aligned}\right.
    & \Rightarrow \\
    & \Rightarrow &
    \left\{\begin{aligned} 
        & \lVert\mathbf{w}\rVert^2+C\cdot\sum_{j=1}^{K}\xi_j\rightarrow\min &\\ 
        & \xi_i=(1-M_i)_+,&\text{ где }i = 1,\dots,K
    \end{aligned}\right.
    & \Rightarrow \\
    & \Rightarrow &
    \begin{aligned} 
        & C\cdot\sum_{j=1}^{K}(1-M_j)_++\lVert\mathbf{w}\rVert^2\rightarrow\min
    \end{aligned}
    & \Rightarrow \\
    & \Rightarrow &
    \begin{aligned} 
        \sum_{j=1}^{K}(1-M_j)_++\frac{1}{C}\lVert\mathbf{w}\rVert^2\rightarrow\min
    \end{aligned}
    &
\end{array}
$$

Итого, есть оптимизационная задача:
$$
\sum_{j=1}^{K}(1-M_j)_++\frac{1}{C}\lVert\mathbf{w}\rVert^2\rightarrow\min
$$

Решать её привычными градиентными методами не получится, т.к. из-за наличия в составе функции $(1-M_j)_+$, график общей функции имеет "острые углы", в которых не существует производной

Но выполнение условий теоремы Куна-Таккера помогают свести эту задачу к поиску седловой точки функции Лагранжа (именно такой алгоритм реализован во многих библиотеках). При сведении нашей задачи к упомянутой ранее получаем полезные соотношения (следствия из необходимых условий седловой точки):
$$
\left\{\begin{aligned} 
    & \mathbf{w} = \sum_{i=1}^{K}\lambda_i \cdot c_i \cdot \mathbf{x}_i \\
    & 0 \leqslant \lambda_i \leqslant C, &\text{ где } i=1,...,K
\end{aligned}\right.
$$

Получается, $\mathbf{w}$ &ndash; линейная комбинация объектов из выборки. Таким образом:
  * Если $\lambda_i=0$, то $\mathbf{x}_i$ не используется для вычисления $\mathbf{w}$
  * Если $\lambda_i\neq0$, то $\mathbf{x}_i$ используется для вычисления $\mathbf{w}$ и называется _**опорным вектором**_

Более подробно классифицируем объекты выборки по $\lambda$, $\xi$ и $M$:
  * $\lambda_i=0;\xi_i=0;M_i\geqslant1$ &ndash; неинформативный объект, лежащий за пределами полосы
  * $0<\lambda_i< C;\xi_i=0;M_i=1$ &ndash; опорный объект, лежащий на границе полосы
  * $\lambda_i=C;\xi_i>0;M_i\leqslant1$ &ndash; ошибочный опорный объект, лежащий не в "своей" области

Если есть основания полагать, что выборка почти линейно разделима, и лишь объекты-выбросы классифицируются неверно, то можно применить фильтрацию выбросов. Сначала задача решается при некотором $C$, и из выборки удаляется небольшая доля объектов, имеющих наибольшую величину ошибки $\xi_i$. После этого задача решается заново по усечённой выборке. Возможно, придётся проделать несколько таких итераций, пока оставшиеся объекты не окажутся линейно разделимыми

### Ядра SVM

Существует ещё один подход к решению проблемы линейной неразделимости. Это переход от исходного пространства признаковых описаний объектов $X$ к новому пространству $H$ с помощью некоторого преобразования $\psi : X \rightarrow H$. Если пространство $H$ имеет достаточно высокую размерность, то можно надеяться, что в нём выборка окажется линейно разделимой. Такое пространство $H$ называется _**спрямляющим**_

Единственным отличием при решении задачи в пространстве $H$ будет замена скалярного произведения $\mathbf{x}\cdot \mathbf{y}$, проводимого в $X$ на $\psi(\mathbf{x})\cdot \psi(\mathbf{y})$, проводимое в $H$.

Таким образом, на пространство $H$ накладывается условие о существовании в нём операции скалярного произведения. В частности, на роль $H$ подойдёт любое евклидово пространство, а в общем случае &ndash; и гильбертово

Поскольку оптимизационная задача
$$
\sum_{j=1}^{K}(1-M_j)_++\frac{1}{C}\lVert\mathbf{w}\rVert^2\rightarrow\min
$$
а конкретно функция
$$
(1-M_j)_+ = \max(0, 1-с_j\cdot(\mathbf{w}\cdot\mathbf{x}_j + b))
$$
зависит только от скалярных произведений объектов, а не от их размерности или того, как они представлены, можно заменить скалярное произведение $\mathbf{x} \cdot \mathbf{y}$ на ядро $K(\mathbf{x}, \mathbf{y})$

_**Ядро**_ &ndash; это некая функция $K : X \times X \rightarrow \mathbb{R}$, представимая в виде $K(\mathbf{x}, \mathbf{y}) = \psi(\mathbf{x})\cdot\psi(\mathbf{y})$ при некотором отображении $\psi : X \rightarrow H$,
где $H$ &ndash; пространство со скалярным произведением

Ядром может быть не всякая функция, а только та, которая удовлетворяет теореме Мерсера:
    
Функция $K(\mathbf{x}, \mathbf{y})$ является ядром тогда и только тогда, когда она симметрична и неотрицательно определена, то есть:
$$
\left\{\begin{aligned} 
    & K(\mathbf{x}, \mathbf{y}) = K(\mathbf{y}, \mathbf{x}) \\
    & \int_X\int_XK(\mathbf{x}, \mathbf{y})g(\mathbf{x})g(\mathbf{y})d\mathbf{x}d\mathbf{y} \geqslant 0 & \forall g: X \rightarrow \mathbb{R}
\end{aligned}\right.
$$

К счастью, класс функций, которые удовлетворяют этой теореме, довольно широк

#### Линейное ядро (Linear)

<img src="https://qu4nt.github.io/sklearn-doc-es/_images/sphx_glr_plot_svm_kernels_001.png" alt="drawing"/>

$$
K(\mathbf{x}, \mathbf{y}) = \mathbf{x} \cdot \mathbf{y}
$$

#### Полиномиальное ядро (Polynomial)

<img src="https://qu4nt.github.io/sklearn-doc-es/_images/sphx_glr_plot_svm_kernels_002.png" alt="drawing"/>

$$
K(\mathbf{x}, \mathbf{y}) = (\gamma \cdot \mathbf{x} \cdot \mathbf{y} + r)^d
$$

#### Гауссовское (RBF)

<img src="https://qu4nt.github.io/sklearn-doc-es/_images/sphx_glr_plot_svm_kernels_003.png" alt="drawing"/>

$$
K(\mathbf{x}, \mathbf{y}) = \exp\left(-\gamma\lVert\mathbf{x} - \mathbf{y}\rVert^2\right)
$$

#### Сигмоидальное (Sigmoig)

$$
K(\mathbf{x}, \mathbf{y}) = \tanh\left(\gamma\mathbf{x} \cdot \mathbf{y} + r\right)
$$

## Программирование

### sklearn

```Python
class sklearn.svm.SVC(*, C, kernel, degree, gamma, coef0, ...)
```

* `C` &ndash; параметр регуляризации (ограничение на $\lambda$), по умолчанию &ndash; `1.0`
* `kernel` &ndash; ядро, по умолчанию &ndash; `rbf`
* `degree` &ndash; степень, используемая для полиномиального ядра, по умолчанию &ndash; `3`
* `gamma` &ndash; $\gamma$, используемая для гауссовского, полиномиального и сигмоидального ядер
* `coef0` &ndash; $r$, используемый в полиномиальном и сигмоидальном ядрах, по умолчанию &ndash; `0.0`

Помимо общего классификатора `sklearn.svm.SVC` библиотека предоставляет классификатор `sklearn.svm.LinearSVC`. Данный классификатор &ndash; более гибко настраиваемая реализация класса `sklearn.svm.SVC` с линейным ядром. Как написано в документации, использование этого классификатора оправдано, если:
  * выборка линейно разделима (использование обычного `sklearn.svm.SVC` не предоставляет возможности более гибкой настройки)
  * выборка имеет очень большой размер (алгоритм с линейным ядром считается намного быстрее, чем с нелинейным)

### Задача определения радиопульсара

<img src="https://minio.nplus1.ru/app-images/173414/21e042b955d6a65f1ef1694564a193bc.jpg" alt="drawing" width=500/>

_**Радиопульсар**_ - космический источник радиоизлучения, приходящего на Землю в виде периодических всплесков (импульсов). Согласно доминирующей астрофизической модели, пульсары представляют собой вращающиеся нейтронные звёзды с магнитным полем, которое наклонено к оси вращения, что вызывает модуляцию приходящего на Землю излучения.

Набор данных содержит 16259 ложных примеров, вызванных интерференцией или шумом, и 1639 реальных примеров радиопульсаров.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [ ]:
DATA_FILE_PATH = "radiopulsars.csv"

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/alexandrehsd/Predicting-Pulsar-Stars/master/pulsar_stars.csv",
    DATA_FILE_PATH)
pass

In [ ]:
df = pd.read_csv(DATA_FILE_PATH)
df.columns = ["IP Mean",     "IP Sd",     "IP Kurtosis",     "IP Skewness", 
              "DM-SNR Mean", "DM-SNR Sd", "DM-SNR Kurtosis", "DM-SNR Skewness",
              "target_class"]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
X = df.drop(["target_class"], axis=1)
y = df["target_class"]

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [ ]:
X.head()

In [ ]:
def draw_boxplot(plot_number: int, column: str) -> None:
    plt.subplot(4, 2, plot_number)
    X.boxplot(column=column)

plt.figure(figsize=(24,20))

for i, column in enumerate(X.columns):
    draw_boxplot(i + 1, column)

In [ ]:
def draw_histplot(plot_number: int, column: str, n_bins: int) -> None:
    plt.subplot(4, 2, plot_number)
    fig = X[column].hist(bins=n_bins)
    fig.set_xlabel(column)

plt.figure(figsize=(24,20))

for i, column in enumerate(X.columns):
    draw_histplot(i + 1, column, 20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
simple_clf = SVC()
simple_clf.fit(X_train, y_train)
f1_score(y_test, simple_clf.predict(X_test))

In [ ]:
clf = SVC()

linear_params = {
    "C":      [1, 5, 10, 50, 100],
    "kernel": ["linear"]
}

poly_params = {
    "C":      [1, 5, 10, 50, 100],
    "kernel": ["poly"],
    "degree": np.arange(1, 6, 1),       # 1, ..., 5
    "gamma":  ["scale", "auto"],
    "coef0":  np.arange(-0.9, 1.2, 0.3) # -0.9, -0.6, ..., 0.6, 0.9
}

rbf_params = {
    "C":      [1, 5, 10, 50, 100],
    "kernel": ["rbf"],
    "gamma":  ["scale", "auto"]
}

sigmoid_params = {
    "C":      [1, 5, 10, 50, 100],
    "kernel": ["sigmoid"],
    "gamma":  ["scale", "auto"],
    "coef0":  np.arange(-0.9, 1.2, 0.3) # -0.9, -0.6, ..., 0.6, 0.9
}

params = [
    linear_params,
    poly_params,
    rbf_params,
    sigmoid_params
]

grid_search = GridSearchCV(estimator=clf,
                           param_grid=params,
                           scoring="f1",
                           n_jobs=8,
                           verbose=10)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

Наилучшие параметры `{'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}`

In [ ]:
best_clf = grid_search.best_estimator_

In [ ]:
f1_score(y_test, best_clf.predict(X_test))

In [ ]:
simple_cm = pd.DataFrame(data=confusion_matrix(y_test, simple_clf.predict(X_test)),
                         columns=["Actual P", "Actual N"], 
                         index=["Predict P", "Predict N"])
best_cm = pd.DataFrame(data=confusion_matrix(y_test, best_clf.predict(X_test)),
                       columns=["Actual P", "Actual N"], 
                       index=["Predict P", "Predict N"])

In [ ]:
sns.heatmap(simple_cm, annot=True, fmt='d', cmap='YlGnBu')
pass

In [ ]:
sns.heatmap(best_cm, annot=True, fmt='d', cmap='YlGnBu')
pass

## Заключение

Преимущества SVM перед методом стохастического градиента и нейронными сетями:
  * Задача выпуклого квадратичного программирования хорошо изучена и имеет единственное решение
  * Метод опорных векторов эквивалентен двухслойной нейронной сети, где число нейронов на скрытом слое определяется автоматически как число опорных векторов
  * Принцип оптимальной разделяющей гиперплоскости приводит к максимизации ширины разделяющей полосы, а следовательно, к более уверенной классификации

Недостатки классического SVM:

  * Неустойчивость к шуму: выбросы в исходных данных становятся опорными объектами-нарушителями и напрямую влияют на построение разделяющей гиперплоскости.
  * Не описаны общие методы построения ядер и спрямляющих пространств, наиболее подходящих для конкретной задачи.
  * Нет отбора признаков.
  * Необходимо подбирать константу C при помощи кросс-валидации.